## 설문조사 단일항목 데이터 전처리

데이터수집 - 전처리 - 분석 - 결과도출 순으로 진행

In [205]:
import math # NaN값 구별때문에 import
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px

# matplotlib 한글 글꼴 설정
plt.rc('font', family='Malgun Gothic')

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 테마 컬러맵 미리 정의
color_scale = ['rgb(5,31,69)',
               'rgb(8,48,107)',
               'rgb(11,62,138)',
               'rgb(8,81,156)',
               'rgb(7,90,176)',
               'rgb(33,113,181)',
               'rgb(66,146,198)',
               'rgb(81,157,207)',
               'rgb(107,174,214)',
               'rgb(158,202,225)',
               'rgb(198,219,239)',
               'rgb(222,235,247)',
               'rgb(247,251,255)']

### 응답자 특성 설문
| `문항` | `보기` |
|:---------:|:---------:|
| <b>`성별`<b> | ①남성    ②여성 |
| <b>`연령`<b>   | ①10대(15세이상)    ②20대   ③30대   ④40대   ⑤50대   ⑥60대이상, 추가설문 : 출생년도(___)년 |
| <b>`학력`<b>   | ①초등학교 졸업(무학 포함)    ②중학교 재학   ③중학교 졸업   ④고등학교 재학   ⑤고등학교 졸업<br>   ⑥대학교 재학    ⑦대학교 졸업    ⑧대학원 재학(석사 또는 박사) ⑨ 대학원 졸업(석사또는 박사) |
| <b>`거주지역`<b>   | ①서울    ②부산   ③대구   ④인천   ⑤광주   ⑥대전    ⑦울산    ⑧세종   ⑨경기<br>   ⑩강원   ⑪충북   ⑫충남   ⑬전북   ⑭전남   ⑮경북   ⑯경남   ⑰제주 |
| <b>`직업`<b>   | ①산림임업종사자    ②사무종사자   ③서비스종사자   ④판매종사자   ⑤기술기능종사자   ⑥단순노무종사자    ⑦전문가(교수 포함)<br>    ⑧경영직종사자   ⑨자영업자(개인사업자)   ⑩공무원(교사포함)   ⑪학생(대학원생포함)   ⑫전업주부   ⑬기타(___) |

In [116]:
# 응답자 특성 딕셔너리
Resp_Character = {
    '성별': {1: '남성', 2: '여성'},
    '연령': {1: '10대(15세이상)', 2: '20대', 3: '30대', 4: '40대', 5: '50대', 6: '60대이상'},
    '학력': {1: '초등학교 졸업(무학 포함)', 2: '중학교 재학', 3: '중학교 졸업', 4: '고등학교 재학', 5: '고등학교 졸업', 6: '대학교 재학', 7: '대학교 졸업', 8: '대학원 재학(석사 또는 박사)', 9: '대학원 졸업(석사또는 박사)'},
    '거주지역': {1: '서울', 2: '부산', 3: '대구', 4: '인천', 5: '광주', 6: '대전', 7: '울산', 8: '세종', 9: '경기', 10: '강원', 11: '충북', 12: '충남', 13: '전북', 14: '전남', 15: '경북', 16: '경남', 17: '제주'},
    '직업': {1: '산림임업종사자', 2: '사무종사자', 3: '서비스종사자', 4: '판매종사자', 5: '기술기능종사자', 6: '단순노무종사자', 7: '전문가(교수 포함)', 8: '경영직종사자', 9: '자영업자(개인사업자)', 10: '공무원(교사포함)', 11: '학생(대학원생포함)', 12: '전업주부', 13: '기타'}}

In [187]:
# 엑셀 파일 읽기
df_common = pd.read_excel('dataset/전국민 산림문화 인식 태도 향유실태 조사_일반국민_230530.xlsx', sheet_name="DATA")

# 산림엑스포 설문데이터 로드 및 응답자 특성 매핑
df_expo = pd.read_excel('dataset/전국민 산림문화 인식 및 태도 향유실태 조사_산림엑스포_230922.xlsx', sheet_name="DATA")
df_expo['SQ1'] = df_expo['SQ1'].replace(Resp_Character['성별'])
df_expo['SQ2_1'] = df_expo['SQ2_1'].replace(Resp_Character['연령'])
df_expo['SQ3'] = df_expo['SQ3'].replace(Resp_Character['학력'])
df_expo['SQ4'] = df_expo['SQ4'].replace(Resp_Character['거주지역'])
df_expo['SQ5'] = df_expo['SQ5'].replace(Resp_Character['직업'])
# 응답자 특성 5번항목의 '기타'항목자 처리
for index, row in df_expo.iterrows():
    if row['SQ5'] == '기타':
        row['SQ5'] = row['SQ5'] + "(%s)"%row['SQ5_13_ETC']
        df_expo.iloc[index] = row['SQ5']

In [91]:
pd.read_excel('dataset/전국민 산림문화 인식 및 태도 향유실태 조사_산림엑스포_230922.xlsx', sheet_name="변수가이드")['문항'].tolist()[75:84]

['문화·예술 관람 행사(예 : 뮤직콘서트, 뮤지컬 갈라쇼 등)',
 '문화·예술 참여 행사(예 : 나무장난감 놀이마당축제 등)',
 '레포츠 행사(예 : 전국 스포츠 클라이밍 대회 등)',
 '학습 행사(예 : 산림과학 학술대회 등)',
 '관광 활동(예 : 설악산 자생식물원, 용대관광단지 등)',
 '전시 관람 활동(예 : 푸른지구관, 산림평화관, 문화유산관 등)',
 '체험 활동(예 : 저탄소 업사이클링 공예체험 등)',
 '사회 활동(예 : 개·폐회식 참여, 인적교류 등) ',
 '기타']

In [273]:
a = ['기타(실내건축)-빈도', '기타(실내건축)-비율', '휴식 및 건강증진-빈도', '휴식 및 건강증진-비율',
       '기타(산림은거)-빈도', '기타(산림은거)-비율', '기타(업무)-빈도', '기타(업무)-비율', '기타(무직)-빈도',
       '기타(무직)-비율', '자연(산림) 감수성함양-빈도', '자연(산림) 감수성함양-비율',
       '산림에 대한 소양증진(지식,정보등 습득)-빈도', '산림에 대한 소양증진(지식,정보등 습득)-비율', '기타(학과행사)-빈도',
       '기타(학과행사)-비율', '창작활동에 대한 기대-빈도', '창작활동에 대한 기대-비율', '즐겁고 재미난 체험-빈도',
       '즐겁고 재미난 체험-비율', '기타(체험학습)-빈도', '기타(체험학습)-비율', '대인관계형성-빈도', '대인관계형성-비율',
       '자기계발-빈도', '자기계발-비율', '가족과의 시간-빈도', '가족과의 시간-비율',
       '지역의 독특한 자연 또는 문화체험-빈도', '지역의 독특한 자연 또는 문화체험-비율']
Q2 = {}
for i,v in enumerate(a):
        Q2[v] = [None]
Q2

{'기타(실내건축)-빈도': [None],
 '기타(실내건축)-비율': [None],
 '휴식 및 건강증진-빈도': [None],
 '휴식 및 건강증진-비율': [None],
 '기타(산림은거)-빈도': [None],
 '기타(산림은거)-비율': [None],
 '기타(업무)-빈도': [None],
 '기타(업무)-비율': [None],
 '기타(무직)-빈도': [None],
 '기타(무직)-비율': [None],
 '자연(산림) 감수성함양-빈도': [None],
 '자연(산림) 감수성함양-비율': [None],
 '산림에 대한 소양증진(지식,정보등 습득)-빈도': [None],
 '산림에 대한 소양증진(지식,정보등 습득)-비율': [None],
 '기타(학과행사)-빈도': [None],
 '기타(학과행사)-비율': [None],
 '창작활동에 대한 기대-빈도': [None],
 '창작활동에 대한 기대-비율': [None],
 '즐겁고 재미난 체험-빈도': [None],
 '즐겁고 재미난 체험-비율': [None],
 '기타(체험학습)-빈도': [None],
 '기타(체험학습)-비율': [None],
 '대인관계형성-빈도': [None],
 '대인관계형성-비율': [None],
 '자기계발-빈도': [None],
 '자기계발-비율': [None],
 '가족과의 시간-빈도': [None],
 '가족과의 시간-비율': [None],
 '지역의 독특한 자연 또는 문화체험-빈도': [None],
 '지역의 독특한 자연 또는 문화체험-비율': [None]}

## Q1. 귀하가 강원세계산림엑스포에 참여하는 참여동기는 무엇입니까? - 빈도분석

In [218]:
# 매핑 딕셔너리 정의
Q1_map = {'1': '휴식 및 건강증진',
        '2': '자기계발',
        '3': '산림에 대한 소양증진(지식,정보등 습득)',
        '4': '즐겁고 재미난 체험',
        '5': '자연(산림) 감수성함양',
        '6': '가족과의 시간',
        '7': '대인관계형성',
        '8': '창작활동에 대한 기대',
        '9': '지역의 독특한 자연 또는 문화체험',
        '10': '기타'}

# 데이터프레임에서 'Q1_1' 열과 'Q1_2' 열의 빈도분석을 수행
frequency1_1 = df_expo['Q1_1'].value_counts().reset_index()
frequency1_1.columns = ['값', '1순위']

frequency1_2 = df_expo['Q1_2'].value_counts().reset_index()
frequency1_2.columns = ['값', '2순위']

# '값' 열의 값을 딕셔너리를 사용하여 일괄적으로 변경하는 함수 정의
def map_val(series, dic):
        lst = []
        for value in series:
                if str(value).isdigit():
                        lst.append(dic[str(value)])
                else:
                        lst.append(value.replace('10', '기타'))
        return lst
        
frequency1_1['값'] = map_val(frequency1_1['값'], Q1_map)
frequency1_2['값'] = map_val(frequency1_2['값'], Q1_map)

In [ ]:
# '값' column을 기준으로 데이터프레임을 병합
frequency1_C = pd.merge(frequency1_1, frequency1_2, on='값', how='outer')
frequency1_C.fillna(0, inplace=True)
frequency1_C['2순위'] = frequency1_C['2순위'].astype(int)
# '1순위'와 '2순위' 빈도를 합산
frequency1_C['1순위+2순위_빈도'] = frequency1_C['1순위'] + frequency1_C['2순위']
frequency1_C = frequency1_C[['값', '1순위+2순위_빈도']]
frequency1_C.sort_values('1순위+2순위_빈도', ascending=False, inplace=True)
frequency1_C

In [225]:
# 빈도를 비율로 변환
frequency1_1['비율'] = frequency1_1['1순위'] / frequency1_1['1순위'].sum()

# 파이 차트 시각화 (크기 조절)
fig = px.pie(frequency1_1, values='비율', names='값', title="<b>귀하가 강원세계산림엑스포에 참여하는 참여동기는 무엇입니까? - 1순위</b>", hole=0.2,
                color_discrete_sequence=color_scale)

fig.update_layout(
    font=dict(size=13),
    height=900,  # 그림의 높이 설정
    width=1400,    # 그림의 너비 설정
    legend=dict(x=1.3, y=0.5)  # 범례 위치 조정 (x, y 조절 가능)
)
# 차트 내에 비율 숫자 직접 표시
pull_lst = [0.01] * 12
fig.update_traces(textinfo='percent+label', pull=pull_lst, insidetextorientation='horizontal')  # 비율 숫자를 라벨과 함께 표시

fig.show()


In [226]:
# 빈도를 비율로 변환
frequency1_2['비율'] = frequency1_2['2순위'] / frequency1_2['2순위'].sum()

# 파이 차트 시각화 (크기 조절)
fig = px.pie(frequency1_2, values='비율', names='값', title="<b>귀하가 강원세계산림엑스포에 참여하는 참여동기는 무엇입니까? - 2순위</b>", hole=0.2,
                color_discrete_sequence=color_scale)
fig.update_layout(
    font=dict(size=13),
    height=900,  # 그림의 높이 설정
    width=1400,    # 그림의 너비 설정
    legend=dict(x=1.3, y=0.5)  # 범례 위치 조정 (x, y 조절 가능)
)
# 차트 내에 비율 숫자 직접 표시
pull_lst = [0.01] * 12
fig.update_traces(textinfo='percent+label', pull=pull_lst, insidetextorientation='horizontal')  # 비율 숫자를 라벨과 함께 표시

fig.show()

In [227]:
# 빈도를 비율로 변환
frequency1_C['비율'] = frequency1_C['1순위+2순위_빈도'] / frequency1_C['1순위+2순위_빈도'].sum()

# 파이 차트 시각화 (크기 조절)
fig = px.pie(frequency1_C, values='비율', names='값', title="<b>귀하가 강원세계산림엑스포에 참여하는 참여동기는 무엇입니까? - 1순위+2순위</b>", hole=0.2,
                color_discrete_sequence=color_scale)
fig.update_layout(
    font=dict(size=13),
    height=900,  # 그림의 높이 설정
    width=1400,    # 그림의 너비 설정
    legend=dict(x=1.3, y=0.5)  # 범례 위치 조정 (x, y 조절 가능)
)
# 차트 내에 비율 숫자 직접 표시
pull_lst = [0.01] * 12
fig.update_traces(textinfo='percent+label', pull=pull_lst, insidetextorientation='horizontal')  # 비율 숫자를 라벨과 함께 표시

fig.show()

### 응답자 특성별 표 정리

In [287]:
combined_list = list(set(frequency1_1['값'].tolist() + frequency1_2['값'].tolist()))
column_lst = []
for i in combined_list:
    column_lst.append("%s-빈도"%i)
    column_lst.append("%s-비율"%i)
resp_map_df = pd.DataFrame(columns = column_lst)

In [293]:
resp_map_df

,기타(실내건축)-빈도,기타(실내건축)-비율,휴식 및 건강증진-빈도,휴식 및 건강증진-비율,기타(산림은거)-빈도,기타(산림은거)-비율,기타(업무)-빈도,기타(업무)-비율,기타(무직)-빈도,기타(무직)-비율,자연(산림) 감수성함양-빈도,자연(산림) 감수성함양-비율,"산림에 대한 소양증진(지식,정보등 습득)-빈도","산림에 대한 소양증진(지식,정보등 습득)-비율",기타(학과행사)-빈도,기타(학과행사)-비율,창작활동에 대한 기대-빈도,창작활동에 대한 기대-비율,즐겁고 재미난 체험-빈도,즐겁고 재미난 체험-비율,기타(체험학습)-빈도,기타(체험학습)-비율,대인관계형성-빈도,대인관계형성-비율,자기계발-빈도,자기계발-비율,가족과의 시간-빈도,가족과의 시간-비율,지역의 독특한 자연 또는 문화체험-빈도,지역의 독특한 자연 또는 문화체험-비율
0,None,None,13,27.1,None,None,3,6.2,None,None,1,2.1,9,18.8,2,4.2,1,2.1,5,10.4,1,2.1,4,8.3,4,8.3,1,2.1,4,8.3


### 1순위+2순위만 할지 아니면 다 할지 물어봐야함

In [289]:
# 'SQ1' 열의 값(성별)으로 필터링
male_rows = df_expo[df_expo['SQ1'] == '남성'].reset_index(drop=True)
frequency_m_1_1 = male_rows['Q1_1'].value_counts().reset_index()
frequency_m_1_1.columns = ['값', '빈도']
frequency_m_1_1['값'] = map_val(frequency_m_1_1['값'], Q1_map)
frequency_m_1_1['비율'] = round((frequency_m_1_1['빈도'] / frequency_m_1_1['빈도'].sum())*100,1)
frequency_m_1_1 = frequency_m_1_1.set_index(keys='값')
frequency_m_1_1

,빈도,비율
값,,
휴식 및 건강증진,13,27.1
"산림에 대한 소양증진(지식,정보등 습득)",9,18.8
즐겁고 재미난 체험,5,10.4
지역의 독특한 자연 또는 문화체험,4,8.3
자기계발,4,8.3
대인관계형성,4,8.3
기타(업무),3,6.2
기타(학과행사),2,4.2
가족과의 시간,1,2.1


In [290]:
resp_map_df.columns

Index(['기타(실내건축)-빈도', '기타(실내건축)-비율', '휴식 및 건강증진-빈도', '휴식 및 건강증진-비율',
       '기타(산림은거)-빈도', '기타(산림은거)-비율', '기타(업무)-빈도', '기타(업무)-비율', '기타(무직)-빈도',
       '기타(무직)-비율', '자연(산림) 감수성함양-빈도', '자연(산림) 감수성함양-비율',
       '산림에 대한 소양증진(지식,정보등 습득)-빈도', '산림에 대한 소양증진(지식,정보등 습득)-비율', '기타(학과행사)-빈도',
       '기타(학과행사)-비율', '창작활동에 대한 기대-빈도', '창작활동에 대한 기대-비율', '즐겁고 재미난 체험-빈도',
       '즐겁고 재미난 체험-비율', '기타(체험학습)-빈도', '기타(체험학습)-비율', '대인관계형성-빈도', '대인관계형성-비율',
       '자기계발-빈도', '자기계발-비율', '가족과의 시간-빈도', '가족과의 시간-비율',
       '지역의 독특한 자연 또는 문화체험-빈도', '지역의 독특한 자연 또는 문화체험-비율'],
      dtype='object')

In [291]:
resp_map = {'기타(실내건축)-빈도': [None],
            '기타(실내건축)-비율': [None],
            '휴식 및 건강증진-빈도': [None],
            '휴식 및 건강증진-비율': [None],
            '기타(산림은거)-빈도': [None],
            '기타(산림은거)-비율': [None],
            '기타(업무)-빈도': [None],
            '기타(업무)-비율': [None],
            '기타(무직)-빈도': [None],
            '기타(무직)-비율': [None],
            '자연(산림) 감수성함양-빈도': [None],
            '자연(산림) 감수성함양-비율': [None],
            '산림에 대한 소양증진(지식,정보등 습득)-빈도': [None],
            '산림에 대한 소양증진(지식,정보등 습득)-비율': [None],
            '기타(학과행사)-빈도': [None],
            '기타(학과행사)-비율': [None],
            '창작활동에 대한 기대-빈도': [None],
            '창작활동에 대한 기대-비율': [None],
            '즐겁고 재미난 체험-빈도': [None],
            '즐겁고 재미난 체험-비율': [None],
            '기타(체험학습)-빈도': [None],
            '기타(체험학습)-비율': [None],
            '대인관계형성-빈도': [None],
            '대인관계형성-비율': [None],
            '자기계발-빈도': [None],
            '자기계발-비율': [None],
            '가족과의 시간-빈도': [None],
            '가족과의 시간-비율': [None],
            '지역의 독특한 자연 또는 문화체험-빈도': [None],
            '지역의 독특한 자연 또는 문화체험-비율': [None]}

for i in resp_map_df.columns:           
    response, method = i.split('-')
    try:
        resp_map[i][0] = frequency_m_1_1.loc[response, method]
    except:
        pass
else:
    resp_map = pd.DataFrame(resp_map)
    resp_map_df = pd.concat([resp_map_df,resp_map])

In [292]:
resp_map_df

,기타(실내건축)-빈도,기타(실내건축)-비율,휴식 및 건강증진-빈도,휴식 및 건강증진-비율,기타(산림은거)-빈도,기타(산림은거)-비율,기타(업무)-빈도,기타(업무)-비율,기타(무직)-빈도,기타(무직)-비율,자연(산림) 감수성함양-빈도,자연(산림) 감수성함양-비율,"산림에 대한 소양증진(지식,정보등 습득)-빈도","산림에 대한 소양증진(지식,정보등 습득)-비율",기타(학과행사)-빈도,기타(학과행사)-비율,창작활동에 대한 기대-빈도,창작활동에 대한 기대-비율,즐겁고 재미난 체험-빈도,즐겁고 재미난 체험-비율,기타(체험학습)-빈도,기타(체험학습)-비율,대인관계형성-빈도,대인관계형성-비율,자기계발-빈도,자기계발-비율,가족과의 시간-빈도,가족과의 시간-비율,지역의 독특한 자연 또는 문화체험-빈도,지역의 독특한 자연 또는 문화체험-비율
0,None,None,13,27.1,None,None,3,6.2,None,None,1,2.1,9,18.8,2,4.2,1,2.1,5,10.4,1,2.1,4,8.3,4,8.3,1,2.1,4,8.3


In [277]:
frequency_m_1_1

,빈도,비율
값,,
휴식 및 건강증진,13,27.1
"산림에 대한 소양증진(지식,정보등 습득)",9,18.8
즐겁고 재미난 체험,5,10.4
지역의 독특한 자연 또는 문화체험,4,8.3
자기계발,4,8.3
대인관계형성,4,8.3
기타(업무),3,6.2
기타(학과행사),2,4.2
가족과의 시간,1,2.1


## Q2 귀하는 강원세계산림엑스포에 누구와 함께 참여하셨습니까? - 빈도분석

In [28]:
Q2_map = {'1': '혼자',
          '2': '가족과 함께',
          '3': '친구 또는 연인과 함께',
          '4': '직장동료와 함께',
          '5': '친목단체 또는 모임에서',
          '6': '학교에서',
          '7': '기타'}

# 데이터프레임에서 'Q2' 열의 빈도 정의
frequency2 = df_expo['Q2'].value_counts().reset_index()
frequency2.columns = ['값', '빈도']

# map_val 함수('값' 열의 값을 딕셔너리를 사용하여 일괄적으로 변경하는 함수)를 사용
frequency2['값'] = map_val(frequency2['값'], Q2_map)

In [87]:
import plotly.express as px

# 빈도를 비율로 변환
frequency2['비율'] = frequency2['빈도'] / frequency2['빈도'].sum()

# 파이 차트 시각화 (크기 조절)
fig = px.pie(frequency2, values='비율', names='값', title="<b>빈도 비율 분석</b>", hole=0.2,
                color_discrete_sequence=color_scale)
fig.update_layout(
        title={
        'text': '귀하는 강원세계산림엑스포에 누구와 함께 참여하셨습니까?',
        'font': {'size': 24}  # 제목의 폰트 크기 설정
    },
    font=dict(size=13),
    height=900,  # 그림의 높이 설정
    width=1400,    # 그림의 너비 설정
    legend=dict(x=0.9, y=0.5)  # 범례 위치 조정 (x, y 조절 가능)
)
# 차트 내에 비율 숫자 직접 표시
pull_lst = [0.01] * 6
fig.update_traces(textinfo='percent+label', pull=pull_lst, insidetextorientation='horizontal')  # 비율 숫자를 라벨과 함께 표시

fig.show()

## Q3 귀하는 강원세계산림엑스포의 가장 기대되는 프로그램이 무엇입니까? - 빈도분석

In [97]:
# 매핑 딕셔너리 정의
Q3_map = {'1': '문화·예술 관람 행사(예 : 뮤직콘서트, 뮤지컬 갈라쇼 등)',
          '2': '문화·예술 참여 행사(예 : 나무장난감 놀이마당축제 등)',
          '3': '레포츠 행사(예 : 전국 스포츠 클라이밍 대회 등)',
          '4': '학습 행사(예 : 산림과학 학술대회 등)',
          '5': '관광 활동(예 : 설악산 자생식물원, 용대관광단지 등)',
          '6': '전시 관람 활동(예 : 푸른지구관, 산림평화관, 문화유산관 등)',
          '7': '체험 활동(예 : 저탄소 업사이클링 공예체험 등)',
          '8': '사회 활동(예 : 개·폐회식 참여, 인적교류 등) ',
          '9': '기타'}

# 데이터프레임에서 'Q1_1' 열과 'Q1_2' 열의 빈도분석을 수행
frequency3_1 = df_expo['Q3_1'].value_counts().reset_index()
frequency3_1.columns = ['값', '1순위']

frequency3_2 = df_expo['Q3_2'].value_counts().reset_index()
frequency3_2.columns = ['값', '2순위']

# '값' 열의 값을 딕셔너리를 사용하여 일괄적으로 변경하는 함수 정의
def map_val(series, dic):
        lst = []
        for value in series:
                if str(value).isdigit():
                        lst.append(dic[str(value)])
                else:
                        lst.append(value.replace('10', '기타'))
        return lst
        
frequency3_1['값'] = map_val(frequency3_1['값'], Q3_map)
frequency3_2['값'] = map_val(frequency3_2['값'], Q3_map)

In [98]:
frequency3_1

,값,1순위
0,"문화·예술 관람 행사(예 : 뮤직콘서트, 뮤지컬 갈라쇼 등)",27
1,"전시 관람 활동(예 : 푸른지구관, 산림평화관, 문화유산관 등)",26
2,체험 활동(예 : 저탄소 업사이클링 공예체험 등),17
3,"관광 활동(예 : 설악산 자생식물원, 용대관광단지 등)",13
4,문화·예술 참여 행사(예 : 나무장난감 놀이마당축제 등),12
5,학습 행사(예 : 산림과학 학술대회 등),4
6,레포츠 행사(예 : 전국 스포츠 클라이밍 대회 등),4
7,"사회 활동(예 : 개·폐회식 참여, 인적교류 등)",2


In [99]:
frequency3_2

,값,2순위
0,문화·예술 참여 행사(예 : 나무장난감 놀이마당축제 등),26
1,"전시 관람 활동(예 : 푸른지구관, 산림평화관, 문화유산관 등)",22
2,체험 활동(예 : 저탄소 업사이클링 공예체험 등),16
3,"관광 활동(예 : 설악산 자생식물원, 용대관광단지 등)",12
4,학습 행사(예 : 산림과학 학술대회 등),11
5,"문화·예술 관람 행사(예 : 뮤직콘서트, 뮤지컬 갈라쇼 등)",9
6,레포츠 행사(예 : 전국 스포츠 클라이밍 대회 등),5
7,"사회 활동(예 : 개·폐회식 참여, 인적교류 등)",4


In [100]:
# '값' column을 기준으로 데이터프레임을 병합
frequency3_C = pd.merge(frequency3_1, frequency3_2, on='값', how='outer')
frequency3_C.fillna(0, inplace=True)
frequency3_C['2순위'] = frequency3_C['2순위'].astype(int)
# '1순위'와 '2순위' 빈도를 합산
frequency3_C['1순위+2순위_빈도'] = frequency3_C['1순위'] + frequency3_C['2순위']
frequency3_C = frequency3_C[['값', '1순위+2순위_빈도']]
frequency3_C.sort_values('1순위+2순위_빈도', ascending=False, inplace=True)
frequency3_C

,값,1순위+2순위_빈도
1,"전시 관람 활동(예 : 푸른지구관, 산림평화관, 문화유산관 등)",48
4,문화·예술 참여 행사(예 : 나무장난감 놀이마당축제 등),38
0,"문화·예술 관람 행사(예 : 뮤직콘서트, 뮤지컬 갈라쇼 등)",36
2,체험 활동(예 : 저탄소 업사이클링 공예체험 등),33
3,"관광 활동(예 : 설악산 자생식물원, 용대관광단지 등)",25
5,학습 행사(예 : 산림과학 학술대회 등),15
6,레포츠 행사(예 : 전국 스포츠 클라이밍 대회 등),9
7,"사회 활동(예 : 개·폐회식 참여, 인적교류 등)",6


In [104]:
# 빈도를 비율로 변환
frequency3_1['비율'] = frequency3_1['1순위'] / frequency3_1['1순위'].sum()

# 파이 차트 시각화 (크기 조절)
fig = px.pie(frequency3_1, values='비율', names='값', title="<b>귀하는 강원세계산림엑스포의 가장 기대되는 프로그램이 무엇입니까? - 1순위</b>", hole=0.2,
                color_discrete_sequence=color_scale)

fig.update_layout(
    font=dict(size=13),
    height=900,  # 그림의 높이 설정
    width=1400,    # 그림의 너비 설정
    legend=dict(x=1.3, y=0.5)  # 범례 위치 조정 (x, y 조절 가능)
)
# 차트 내에 비율 숫자 직접 표시
pull_lst = [0.01] * 12
fig.update_traces(textinfo='percent+label', pull=pull_lst, insidetextorientation='horizontal')  # 비율 숫자를 라벨과 함께 표시

fig.show()


In [111]:
# 빈도를 비율로 변환
frequency3_2['비율'] = frequency3_2['2순위'] / frequency3_2['2순위'].sum()

# 파이 차트 시각화 (크기 조절)
fig = px.pie(frequency3_2, values='비율', names='값', title="<b>귀하는 강원세계산림엑스포의 가장 기대되는 프로그램이 무엇입니까? - 2순위</b>", hole=0.2,
                color_discrete_sequence=color_scale)
fig.update_layout(
    font=dict(size=13),
    height=900,  # 그림의 높이 설정
    width=1400,    # 그림의 너비 설정
    legend=dict(x=1.3, y=1.1)  # 범례 위치 조정 (x, y 조절 가능)
)
# 차트 내에 비율 숫자 직접 표시
pull_lst = [0.01] * 12
fig.update_traces(textinfo='percent+label', pull=pull_lst, insidetextorientation='horizontal')  # 비율 숫자를 라벨과 함께 표시

fig.show()

In [113]:
# 빈도를 비율로 변환
frequency3_C['비율'] = frequency3_C['1순위+2순위_빈도'] / frequency3_C['1순위+2순위_빈도'].sum()

# 파이 차트 시각화 (크기 조절)
fig = px.pie(frequency3_C, values='비율', names='값', title="<b>귀하는 강원세계산림엑스포의 가장 기대되는 프로그램이 무엇입니까? - 1순위+2순위</b>", hole=0.2,
                color_discrete_sequence=color_scale)
fig.update_layout(
    font=dict(size=13),
    height=900,  # 그림의 높이 설정
    width=1400,    # 그림의 너비 설정
    legend=dict(x=1.3, y=1.1)  # 범례 위치 조정 (x, y 조절 가능)
)
# 차트 내에 비율 숫자 직접 표시
pull_lst = [0.01] * 12
fig.update_traces(textinfo='percent+label', pull=pull_lst, insidetextorientation='horizontal')  # 비율 숫자를 라벨과 함께 표시

fig.show()